<a href="https://colab.research.google.com/github/2kunal6/UniBonn/blob/master/cuda_lab/2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import torch
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim

import matplotlib.pyplot as plt
import numpy as np

np.random.seed(1)

transform_list = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.ToTensor(),
            transforms.Normalize([0.5,],[0.5,])
        ])
train = torch.utils.data.DataLoader(
    datasets.CIFAR10("./sample_data", train=True, transform=transform_list, download=True), batch_size = 32, shuffle = True
)

test = torch.utils.data.DataLoader(
    datasets.CIFAR10("./sample_data", train=False, transform=transform_list, download=True), batch_size = 32, shuffle = True
)

print("datasets loaded")

IMG_SIZE = 32*32

for data in train:
  X, y = data[0], data[1]
  #plt.imshow(X[0])
  #print("Label: " + str(y[0]))
  #plt.show()
  break

class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.linear(x)

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

model = LogisticRegression(IMG_SIZE, 10).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(20):
  for data in train:
    X, y = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    output = model(X.view(-1, IMG_SIZE))
    loss = F.cross_entropy(output, y)
    loss.backward()
    optimizer.step()
  print(f'epoch: {epoch}, loss: {loss}')

total = 0
correct = 0

with torch.no_grad():
  for data in test:
    X, y = data[0].to(device), data[1].to(device)
    output = model(X.view(-1, IMG_SIZE))
    _, predicted = torch.max(output, 1)
    total += y.size(0)
    correct += (predicted == y).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified
datasets loaded
Running on the GPU
epoch: 0, loss: 2.064054250717163
epoch: 1, loss: 3.618572235107422
epoch: 2, loss: 2.8538265228271484
epoch: 3, loss: 2.2574737071990967
epoch: 4, loss: 2.272906541824341
epoch: 5, loss: 1.9147344827651978
epoch: 6, loss: 2.237785577774048
epoch: 7, loss: 3.054563283920288
epoch: 8, loss: 2.49027681350708
epoch: 9, loss: 2.8396451473236084
epoch: 10, loss: 1.7107367515563965
epoch: 11, loss: 2.2247610092163086
epoch: 12, loss: 3.644300699234009
epoch: 13, loss: 2.730564594268799
epoch: 14, loss: 3.289738416671753
epoch: 15, loss: 2.9807775020599365
epoch: 16, loss: 2.134479522705078
epoch: 17, loss: 3.254472017288208
epoch: 18, loss: 2.8755292892456055
epoch: 19, loss: 2.839099168777466
Accuracy of the network on the 10000 test images: 21 %
